# Unicorn Companies
A unicorn company is a privately held company with a current valuation of over $1 billion USD. This dataset consists of unicorn companies and startups across the globe as of November 2021, including country of origin, sector, select investors, and valuation of each unicorn. 

_Note former unicorn companies that have since exited due to IPO or acquisitions are not included in this list._

Not sure where to begin? Scroll to the bottom to find challenges!

**SCENERIO QUESTION**

You have been hired as a data scientist for a company that invests in start-ups. Your manager is interested in whether it is possible to predict whether a company reaches a valuation over 5 billion based on characteristics such as its country of origin, its category, and details about its investors.

Using the dataset provided, you have been asked to test whether such predictions are possible, and the confidence one can have in the results.

You will need to prepare a report that is accessible to a broad audience. It will need to outline your motivation, steps, findings, and conclusions.

In [1]:


--This code examines the performance of each industries per country

Select  industry,
country,
count(c.company_id) no_of_companies,
ROUND(AVG(valuation),2) avg_valuation,
ROUND(AVG(funding),2) Average_funding,
MAX(valuation) max_valuation,
MIN(valuation) min_valuation,
SUM(valuation) country_industry_val,
MAX(funding) max_funding, 
MIN(funding) min_funding,
SUM(funding) industry_gross_funding
From companies c
JOIN industries i
ON c.company_id = i.company_id
JOIN funding f
ON i.company_id = f.company_id
GROUP by 2,1
Order by 6 desc;



,industry,country,no_of_companies,avg_valuation,average_funding,max_valuation,min_valuation,country_industry_val,max_funding,min_funding,industry_gross_funding
0,Artificial intelligence,China,20,1.070000e+10,9.122500e+08,180000000000,1000000000,214000000000,8000000000,79000000,18245000000
1,Other,United States,30,6.500000e+09,9.173667e+08,100000000000,1000000000,195000000000,7000000000,0,27521000000
2,E-commerce & direct-to-consumer,China,29,6.448276e+09,8.963103e+08,100000000000,1000000000,187000000000,5000000000,96000000,25993000000
3,Fintech,United States,120,3.783333e+09,4.168917e+08,95000000000,1000000000,454000000000,2000000000,0,50027000000
4,Fintech,Sweden,1,4.600000e+10,4.000000e+09,46000000000,46000000000,46000000000,4000000000,4000000000,4000000000
...,...,...,...,...,...,...,...,...,...,...,...
189,Other,Japan,2,1.000000e+09,3.300000e+08,1000000000,1000000000,2000000000,520000000,140000000,660000000
190,Data management & analytics,Israel,1,1.000000e+09,2.640000e+08,1000000000,1000000000,1000000000,264000000,264000000,264000000
191,E-commerce & direct-to-consumer,Hong Kong,1,1.000000e+09,1.400000e+08,1000000000,1000000000,1000000000,140000000,140000000,140000000
192,E-commerce & direct-to-consumer,Norway,2,1.000000e+09,3.030000e+08,1000000000,1000000000,2000000000,337000000,269000000,606000000


[Source](https://www.cbinsights.com/research-unicorn-companies) of dataset. 

In [1]:
Select  c.company,industry,
country,valuation

From companies c
JOIN industries i
ON c.company_id = i.company_id
JOIN funding f
ON i.company_id = f.company_id
--GROUP by 2,1
Order by 4 desc;



,company,industry,country,valuation
0,Bytedance,Artificial intelligence,China,180000000000
1,SpaceX,Other,United States,100000000000
2,SHEIN,E-commerce & direct-to-consumer,China,100000000000
3,Stripe,Fintech,United States,95000000000
4,Klarna,Fintech,Sweden,46000000000
...,...,...,...,...
1069,UISEE Technology,"Supply chain, logistics, & delivery",China,1000000000
1070,Konfio,Fintech,Mexico,1000000000
1071,Shippo,"Supply chain, logistics, & delivery",United States,1000000000
1072,Gymshark,E-commerce & direct-to-consumer,United Kingdom,1000000000


![image-7](image-7.png)


In [3]:
/*Checking the global industry average valuation and total valuation to compare country performance to global metrics*/

Select industry,count(i.company_id),
ROUND(AVG(valuation),2) global_avg_valuation,
SUM(valuation) tot_global_val
FROM industries i
JOIN funding f
ON i.company_id = f.company_id
GROUP BY 1
ORDER BY 3 DESC;

/* Next, using CTEs I will compare the country performance with global and rank each country per industry to 
  identify the countries with best industry categories */
  
WITH country_index
AS (Select  industry,
country,
count(c.company_id) no_of_companies,
ROUND(AVG(valuation),2) avg_valuation,
ROUND(AVG(funding),2) average_funding,
MAX(valuation) max_valuation,
MIN(valuation) min_valuation,
SUM(valuation) country_industry_val,
MAX(funding) max_funding, 
MIN(funding) min_funding,
SUM(funding) industry_total_funding
From companies c
JOIN industries i
ON c.company_id = i.company_id
JOIN funding f
ON i.company_id = f.company_id
GROUP by 2,1
Order by 6 desc),

global_index AS
(Select industry,count(i.company_id),
ROUND(AVG(valuation),2) global_avg_valuation,
SUM(valuation) tot_global_val
FROM industries i
JOIN funding f
ON i.company_id = f.company_id
GROUP BY 1
ORDER BY 3 DESC)

SELECT rank_filter.* FROM
(SELECT c.industry,
c.country,
c.no_of_companies,
c.avg_valuation,
c.country_industry_val,
g.tot_global_val,
g.global_avg_valuation, 
ROUND((c.country_industry_val *100)/g.tot_global_val,2) AS perc_global_val,
RANK()OVER(PARTITION BY c.industry ORDER BY (c.country_industry_val *100)/g.tot_global_val  DESC)

FROM country_index c
JOIN global_index g
USING(industry)
ORDER BY perc_global_val DESC) rank_filter WHERE RANK <= 3;



,industry,country,no_of_companies,avg_valuation,country_industry_val,tot_global_val,global_avg_valuation,perc_global_val,rank
0,Data management & analytics,United States,31,3.516129e+09,109000000000,136000000000,3.317073e+09,80.15,1
1,Other,United States,30,6.500000e+09,195000000000,252000000000,4.344828e+09,77.38,1
2,Cybersecurity,United States,39,2.538462e+09,99000000000,129000000000,2.580000e+09,76.74,1
3,Internet software & services,United States,149,2.979866e+09,444000000000,595000000000,2.902439e+09,74.62,1
4,Health,United States,51,2.862745e+09,146000000000,198000000000,2.675676e+09,73.74,1
5,Artificial intelligence,China,20,1.070000e+10,214000000000,377000000000,4.488095e+09,56.76,1
6,Consumer & retail,United States,11,5.363636e+09,59000000000,106000000000,4.240000e+09,55.66,1
7,Hardware,China,17,3.176471e+09,54000000000,99000000000,2.911765e+09,54.55,1
8,Fintech,United States,120,3.783333e+09,454000000000,882000000000,3.937500e+09,51.47,1
9,Auto & transportation,China,17,2.941176e+09,50000000000,99000000000,3.193548e+09,50.51,1


When I ranked each country by its percentage of average global valuation, countries with only one company were mostly returned leaving out the ones with more companies but with higher total valuation. 
Hence I used the total valuation for each indusrtry per country and how it compares with the global valuation. 
I used function RANK_FILTER.* to filter my ranking return only the top 3 positions across all industries.

In [4]:
--Next I will rank by percentage average valuation per industry for every country 

WITH country_index
AS (Select  industry,
country,
count(c.company_id) no_of_companies,
ROUND(AVG(valuation),2) avg_valuation,
ROUND(AVG(funding),2) average_funding,
MAX(valuation) max_valuation,
MIN(valuation) min_valuation,
SUM(valuation) country_industry_val,
MAX(funding) max_funding, 
MIN(funding) min_funding,
SUM(funding) industry_total_funding
From companies c
JOIN industries i
ON c.company_id = i.company_id
JOIN funding f
ON i.company_id = f.company_id
GROUP by 2,1
Order by 6 desc),

global_index AS
(Select industry,count(i.company_id),
ROUND(AVG(valuation),2) global_avg_valuation,
SUM(valuation) tot_global_val
FROM industries i
JOIN funding f
ON i.company_id = f.company_id
GROUP BY 1
ORDER BY 3 DESC)

SELECT rank_filter.* FROM
(SELECT c.industry,
c.country,
c.no_of_companies,
c.avg_valuation,
c.country_industry_val,
g.tot_global_val,
g.global_avg_valuation, 
ROUND((c.avg_valuation *100)/g.global_avg_valuation,2) AS perc_global_avg_val,
RANK()OVER(PARTITION BY c.industry ORDER BY (c.avg_valuation *100)/g.global_avg_valuation  DESC)

FROM country_index c
JOIN global_index g
USING(industry)
WHERE c.no_of_companies >= 3
 --(including this WHERE condition removes India which is the best in Travels, 
-- its avg is less than the industry avg)
ORDER BY perc_global_avg_val DESC) rank_filter WHERE RANK = 1;

,industry,country,no_of_companies,avg_valuation,country_industry_val,tot_global_val,global_avg_valuation,perc_global_avg_val,rank
0,Internet software & services,Australia,5,9.200000e+09,46000000000,595000000000,2.902439e+09,316.97,1
1,Artificial intelligence,China,20,1.070000e+10,214000000000,377000000000,4.488095e+09,238.41,1
2,E-commerce & direct-to-consumer,China,29,6.448276e+09,187000000000,426000000000,3.837838e+09,168.02,1
3,Other,United States,30,6.500000e+09,195000000000,252000000000,4.344828e+09,149.60,1
4,Edtech,India,6,5.166667e+09,31000000000,100000000000,3.571429e+09,144.67,1
5,Fintech,United Kingdom,26,5.653846e+09,147000000000,882000000000,3.937500e+09,143.59,1
6,Cybersecurity,Canada,3,3.333333e+09,10000000000,129000000000,2.580000e+09,129.20,1
7,Mobile & telecommunications,United States,12,3.000000e+09,36000000000,89000000000,2.342105e+09,128.09,1
8,"Supply chain, logistics, & delivery",United States,20,3.950000e+09,79000000000,177000000000,3.105263e+09,127.20,1
9,Consumer & retail,United States,11,5.363636e+09,59000000000,106000000000,4.240000e+09,126.50,1


From the above query,I ranked the industries per country based on the average valuation compared to the global average. This will help identify the actual performance of industries where we have at least 3 companies. From the previous ranking USA & CHINA dominated the top 20 having 95% of the biggest industries.

Meanwhile, using average industry valuation, other countries like (AUSTRALIA,INDIA,CANADA,UNITED KINGDOM) came into the list ranking first in (Internet software & services, Edtech, Cybersecurity, and Fintech) respectively.

![image-11](image-11.png)


In [4]:
-- List the 20 most valued companies

select c.company,
f.valuation, country,industry
from companies c
join funding f
on c.company_id =f.company_id
join industries i
on f.company_id = i.company_id
--where industry = 'Artificial intelligence'
group by 1,2,3,4

order by valuation desc
Limit 20;

,company,valuation,country,industry
0,Bytedance,180000000000,China,Artificial intelligence
1,SpaceX,100000000000,United States,Other
2,SHEIN,100000000000,China,E-commerce & direct-to-consumer
3,Stripe,95000000000,United States,Fintech
4,Klarna,46000000000,Sweden,Fintech
5,Canva,40000000000,Australia,Internet software & services
6,Checkout.com,40000000000,United Kingdom,Fintech
7,Instacart,39000000000,United States,"Supply chain, logistics, & delivery"
8,JUUL Labs,38000000000,United States,Consumer & retail
9,Databricks,38000000000,United States,Data management & analytics


In [1]:
--Next, we calculate our measures of central tendency to understand the spread of valuation across all industries.

SELECT industry,
AVG(valuation),
MAX(valuation),MIN(valuation),
PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY valuation),
PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY valuation) AS Third_percentile,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY valuation) AS first_percentile,STDDEV(valuation)
from industries i
join funding f
on i.company_id = f.company_id
group by 1
order by 2 desc;



,industry,avg,max,min,percentile_cont,third_percentile,first_percentile,stddev
0,Artificial intelligence,4.488095e+09,180000000000,1000000000,2000000000,2250000000,1000000000,19512426716
1,Other,4.344828e+09,100000000000,1000000000,2000000000,2000000000,1000000000,13520032861
2,Consumer & retail,4.240000e+09,38000000000,1000000000,2000000000,2000000000,1000000000,7833049640
3,Fintech,3.937500e+09,95000000000,1000000000,2000000000,3000000000,1000000000,8263357172
4,E-commerce & direct-to-consumer,3.837838e+09,100000000000,1000000000,2000000000,3000000000,1000000000,9985936056
5,Edtech,3.571429e+09,22000000000,1000000000,2000000000,3000000000,1000000000,4932346534
6,Data management & analytics,3.317073e+09,38000000000,1000000000,2000000000,3000000000,1000000000,5884042082
7,Travel,3.285714e+09,10000000000,1000000000,2000000000,4500000000,1000000000,3172685624
8,Auto & transportation,3.193548e+09,11000000000,1000000000,2000000000,4000000000,1000000000,2880038829
9,"Supply chain, logistics, & delivery",3.105263e+09,39000000000,1000000000,1000000000,3000000000,1000000000,5780769260


In [5]:
--This code refines our last query to give more details of industries in each country.

SELECT industry,Count(c.company) company_count, 
c.country,
ROUND(AVG(valuation),2) avg_valuation,
MAX(valuation) max_valuation,
MIN(valuation) min_valuation,
PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY valuation) median_valuation,
PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY valuation) AS Third_quartile,
PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY valuation) AS first_quartile,
STDDEV(valuation)
from industries i
join funding f
on i.company_id = f.company_id
join companies c
on f.company_id = c.company_id
group by 1,3
order by 2 desc;

,industry,company_count,country,avg_valuation,max_valuation,min_valuation,median_valuation,third_quartile,first_quartile,stddev
0,Internet software & services,149,United States,2.979866e+09,18000000000,1000000000,2000000000,4000000000,1000000000,2.971645e+09
1,Fintech,120,United States,3.783333e+09,95000000000,1000000000,2000000000,3000000000,1000000000,9.028604e+09
2,Health,51,United States,2.862745e+09,13000000000,1000000000,2000000000,3500000000,1000000000,2.771423e+09
3,Artificial intelligence,46,United States,2.934783e+09,12000000000,1000000000,2000000000,3750000000,1000000000,2.815845e+09
4,Cybersecurity,39,United States,2.538462e+09,9000000000,1000000000,2000000000,3000000000,1000000000,2.292392e+09
...,...,...,...,...,...,...,...,...,...,...
189,Other,1,Brazil,3.000000e+09,3000000000,3000000000,3000000000,3000000000,3000000000,NaN
190,Other,1,Canada,1.000000e+09,1000000000,1000000000,1000000000,1000000000,1000000000,NaN
191,Other,1,Colombia,1.000000e+09,1000000000,1000000000,1000000000,1000000000,1000000000,NaN
192,Other,1,Israel,2.000000e+09,2000000000,2000000000,2000000000,2000000000,2000000000,NaN


In [5]:
--This code removes the outlier company from Artificial intelligence and then returns the industry average 

Select  industry, AVG(valuation) industry_avg_valuation
From funding f
Join companies c
On c.company_id = f.company_id 
Join industries i
On f.company_id =i.company_id 
Where
c.company <> 'Bytedance'
AND c.company <>'SpaceX'
AND c.company <>'Epic Games'

Group by 1
order by 2 desc;

,industry,industry_avg_valuation
0,Consumer & retail,4.240000e+09
1,Fintech,3.937500e+09
2,E-commerce & direct-to-consumer,3.837838e+09
3,Edtech,3.571429e+09
4,Data management & analytics,3.317073e+09
5,Travel,3.285714e+09
6,Auto & transportation,3.193548e+09
7,"Supply chain, logistics, & delivery",3.105263e+09
8,Hardware,2.911765e+09
9,Internet software & services,2.902439e+09


The graphs below uses measures of central tendency to visualize the distribution of company valuation across the four industries to have a feel of valuation data spread. 

![image-3](image-3.png)
![image-4](image-4.png)
![image-5](image-5.png)
![image-6](image-6.png)


Artificial intelligence industry recorded the highest average valuation among the Unicorn companies. It is observed that the valuation for artificial intelligence has an outlier effect from Bytedance a China based company with a valuation of 180 Billion USD which impacted significantly on the industry average. The distribution is positively skewed,with median at 2 billion USD which implies that half of the artificial intelligence companies have a valuation of $2Billion.

Removing the outlier from Artificial intelligence industries we have the mean drop to 2.37 billion USD which is closer to the median. Similarly we filtered 'Other' category to remove two outliers (SpaceX and Epic Games) which also saw the mean drop from 4.3 billion USD to 2.14 billion USD. 

The top 20 performing companies in Artificial intelligence are situated in the USA and CHINA. 
Fifteen(14) companies are from USA while four(4) are CHINESE based and only one each from SINGAPORE and UNITED KINGDOM.

We also have an interesting scenerio in the Fintech industry with USA having the an outlier company worth 95 billion USD, while the industry average is less than 5 billion USD. United Kindom has the highest total valuation in Fintech with its industry average at 5.6 billion  USD and a total of 26 unicorn companies. 



In [1]:
/* Code to extract the names of investors in the Unicorn companies*/

WITH sub1 
 AS
 (Select company, country, valuation,funding,select_investors,
SPLIT_PART (select_investors,',',1) Investor1,
 SPLIT_PART (select_investors,',',2) Investor2,
 SPLIT_PART (select_investors,',',3) Investor3,
 SPLIT_PART (select_investors,',',4) Investor4
from 
(select company,country,
valuation,select_investors,funding,
rank()over(order by valuation desc)
from companies c
join funding f
on c.company_id =f.company_id
join industries i
on f.company_id = i.company_id
where valuation >= 5000000000 
) as Ranking
order by valuation desc)

SELECT company,valuation,investor1
from sub1
UNION  
SELECT company,valuation,investor2
from sub1
UNION 
SELECT company,valuation,investor3
from sub1
UNION 
SELECT company,valuation,investor4
from sub1
order by valuation desc;



,investor1,total_investment,avg_investment,count
0,Sina Weibo,180000000000,1.800000e+11,1
1,SIG Asia Investments,180000000000,1.800000e+11,1
2,Draper Fisher Jurvetson,100000000000,1.000000e+11,1
3,Rothenberg Ventures,100000000000,1.000000e+11,1
4,LowercaseCapital,95000000000,9.500000e+10,1
...,...,...,...,...
247,Borui Capital,6000000000,6.000000e+09,1
248,Point Nine Capital,6000000000,6.000000e+09,1
249,Technology Crossover Ventures,12000000000,6.000000e+09,2
250,Benchmark,11000000000,5.500000e+09,2


In [7]:

/*I used aggregate function with case statment and partition by to group the companies into less than or above $5 billion valuation
 then I used CTE with row number to filter for the industry per country*/
  
  SELECT *
  FROM 
   ( With cte as 
(Select Industry,c.country,
Count(i.company_id) Over(partition by country, industry) tot_company, 
Sum(Case when valuation >= 5000000000 then 1 else 0 end) Over(Partition by country, industry ) above_5B, 
Sum(case when valuation < 5000000000 then 1 else 0 end) Over (partition by country, industry) less_5B
 
From industries i
Join funding f
On i.company_id =f.company_id
Join companies c
On f.company_id = c.company_id) 

Select industry, country,tot_company, 
above_5B/tot_company :: NUMERIC as probability, 
row_number() over(Partition by country,industry Order by country  )  
From cte) prob3
where row_number between 0 and 1

    

,industry,country,tot_company,probability,row_number
0,Fintech,Argentina,1,0.000000,1
1,E-commerce & direct-to-consumer,Australia,1,0.000000,1
2,Fintech,Australia,2,0.500000,1
3,Internet software & services,Australia,5,0.200000,1
4,Edtech,Austria,1,0.000000,1
...,...,...,...,...,...
189,Other,United States,30,0.133333,1
190,"Supply chain, logistics, & delivery",United States,20,0.100000,1
191,Travel,United States,1,1.000000,1
192,Fintech,Vietnam,1,0.000000,1


![image-13](image-13.png)


In [10]:

  /* To get an insight from the investors in companies with valuation over $5B, I have sorted and used split function for the last query, next the columns are collapse together using subquery and then union the table to itself selecting each investor column each time and then  can count the investors.*/

SELECT investor1,SUM(valuation) total_investment,ROUND (AVG(valuation),2) avg_investment, count(*)
FROM
(
WITH sub1 
 AS
 (Select company, country, valuation,funding,select_investors,
 SPLIT_PART (select_investors,',',1)  Investor1,
 SPLIT_PART (select_investors,',',2)  Investor2,
 SPLIT_PART (select_investors,',',3)  Investor3,
 SPLIT_PART (select_investors,',',4)  Investor4
from 
(select company,country,
valuation,select_investors,funding,
rank()over(order by valuation desc)
from companies c
join funding f
on c.company_id =f.company_id
join industries i
on f.company_id = i.company_id
where valuation >= 5000000000 
) as Ranking
order by valuation desc)
    
SELECT company,valuation,investor1
from sub1
UNION ALL
SELECT company,valuation,investor2
from sub1
UNION ALL
SELECT company,valuation,investor3
from sub1
UNION ALL
SELECT company,valuation,investor4
from sub1
order by valuation desc) AS investor_list

GROUP BY investor1
HAVING AVG(valuation) > 5000000000  
ORDER BY avg_investment DESC;

,investor1,total_investment,avg_investment,count
0,Sina Weibo,180000000000,1.800000e+11,1
1,SIG Asia Investments,180000000000,1.800000e+11,1
2,Draper Fisher Jurvetson,100000000000,1.000000e+11,1
3,Rothenberg Ventures,100000000000,1.000000e+11,1
4,LowercaseCapital,95000000000,9.500000e+10,1
...,...,...,...,...
247,Borui Capital,6000000000,6.000000e+09,1
248,Point Nine Capital,6000000000,6.000000e+09,1
249,Technology Crossover Ventures,12000000000,6.000000e+09,2
250,Benchmark,11000000000,5.500000e+09,2


In [9]:
SELECT company,
valuation,
industry,
c.country,
ROUND(AVG(valuation) OVER(PARTITION BY country ORDER BY industry),2) avg_valuation,
SUM(valuation) OVER(PARTITION BY country ORDER BY industry) running_sum,
MAX(valuation) OVER(PARTITION BY country ORDER BY industry)  max_valuation,

from industries i
join funding f
on i.company_id = f.company_id
join companies c
on f.company_id = c.company_id;

,company,valuation,industry,country,avg_valuation,running_sum,max_valuation,min_valuation
0,Uala,2000000000,Fintech,Argentina,2.000000e+09,2000000000,2000000000,2000000000
1,Pet Circle,1000000000,E-commerce & direct-to-consumer,Australia,1.000000e+09,1000000000,1000000000,1000000000
2,Immutable,3000000000,Fintech,Australia,3.333333e+09,10000000000,6000000000,1000000000
3,Airwallex,6000000000,Fintech,Australia,3.333333e+09,10000000000,6000000000,1000000000
4,Canva,40000000000,Internet software & services,Australia,7.000000e+09,56000000000,40000000000,1000000000
...,...,...,...,...,...,...,...,...
1069,Loadsmart,1000000000,"Supply chain, logistics, & delivery",United States,3.433155e+09,1926000000000,100000000000,1000000000
1070,STORD,1000000000,"Supply chain, logistics, & delivery",United States,3.433155e+09,1926000000000,100000000000,1000000000
1071,TripActions,7000000000,Travel,United States,3.439502e+09,1933000000000,100000000000,1000000000
1072,MoMo,2000000000,Fintech,Vietnam,2.000000e+09,2000000000,2000000000,2000000000


In [3]:
/* This code uses case statement to filter the conditions for counting the number of
industries with less than $5 bllion valuation based on date joined and year founded. */


Select  COUNT(f.company_id) company_count,DATE_TRUNC ('year',date_joined) joindate,
case when valuation >= 5000000000 then '1' 
else  '0' end as condition
from funding f
join dates d
ON f.company_id = d.company_id
group by joindate,condition
order by joindate desc;


Select  COUNT(f.company_id) count_company,year_founded,
case when valuation >= 5000000000 then '1' 
else  '0' end as condition
from funding f
join dates d
ON f.company_id = d.company_id
group by condition,year_founded
order by year_founded desc;



,count_company,year_founded,condition
0,11,2021,0
1,3,2020,1
2,22,2020,0
3,42,2019,0
4,3,2019,1
5,10,2018,1
6,51,2018,0
7,10,2017,1
8,64,2017,0
9,98,2016,0


Now we have the list of companies with more than $5B valuation and also the countries with most valued industry. 
We also have the list of investors with average valuation of companies invested and count of the companies.

The companies that joined the unicorn league from year 2014 - 2020 have higher probability of achieving up to $5billion valuation. The probability ranges between 14% - 30% based on the chart below, comparing their percentages to total listed company for those years.

There is a growth in the number of companies founded between 2008 - 2018 which also showed a corresponding increase in the number of those with valuation of at least $5 billion.

The best investment option will be made in countries where the industry is within the top 3   industries as shown in the charts. Also the year joined and year founded is another insight to guide our investment options. Finally, looking at number of the investor, there is no much risk as over 200 investors have invested in companies with average of over $5 billion. 

![image-9](image-9.png)
![image-10](image-10.png)
